In [ ]:
!pip install sentinelhub

# Optimale NDVI threshold analyse. 

Het doel van deze analyse is om de optimale NDVI-drempel te bepalen om onze binaire maskers te genereren.

### Maskers
Een masker is een binair raster met 0 en 1-pixelwaarden. Maskers worden gebruikt om bepaalde pixels buiten beeldverwerking te houden en om beeldstatistieken te berekenen.
Het proces wordt meestal gedaan om "object" of voorgrondpixels te scheiden van achtergrondpixels om te helpen bij beeldverwerking.

We gebruiken maskers om ons model te trainen om bebossing te onderscheiden in een bepaald satellietbeeld. Om onze maskers te maken, gebruiken we NDVI-waarden en de Thresholding-methode.

Om onze maskers te maken, hebben we een optimale NDVI-drempel nodig waarmee we bos en geen bos kunnen onderscheiden in onze multitemporele satellietbeelden.

### Stappenplan
* Stap Voor 1 a 2 geolocaties( 1 a 2 satellietbeelden), satellietbeelden labelen met Labelbox
* Masks genereren van geolocaties voor verschillende NDVI thresholds 
* Verschil bepalen tussen NDVI masks uit Sentinel hub en gelabelde masks
* Optimale NDVI threshold bepalen

In [ ]:
# connect to google drive
from google.colab import drive, files
drive.mount("/content/drive")

In [ ]:
#Import libraries
from PIL import  Image, ImageEnhance
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline
import cv2
from imageio import imread

## Verkrijg gelabelde afbeeldingen van Labelbox

Er waren een paar afbeeldingen in ware kleuren geselecteerd die we mochten labelen. De afbeeldingen werden gelabeld en geannoteerd via Labelbox. In de volgende code zullen we deze afbeeldingen verwerven en plotten. Bovendien zullen we deze gelabelde maskers toevoegen aan een lijst om te gebruiken voor analyse. 

In [ ]:
path = '/content/drive/My Drive/BB8/West_Bengal_15_geolocations_v2/labels_3_images.json'
with open(path) as file:
    data = json.load(file)

In [ ]:
df = pd.read_csv('/content/drive/My Drive/BB8/West_Bengal_15_geolocations_v2/labels_3_images.csv')
df['External ID'][3]

In [ ]:
import pprint
pprint.pprint(data[0]['Labeled Data'])

In [ ]:
mask = data[0]['Label']['objects'][0]['instanceURI']
pprint.pprint(mask)

Get mask for all 4 images

In [ ]:
labeled_masks = []
for i in range(4):
  mask = data[i]['Label']['objects'][0]['instanceURI']# get mask 
  mask = imread(mask)
  mask = np.delete(mask, np.s_[:3], 2)
  mask = np.reshape(mask, (512,512))
  labeled_masks.append(mask)
  masks_ = Image.fromarray(mask)
  print(f"Mask {i}")
  # display(plt.imshow(mask, cmap ='gray'))
  display(masks_)


## Verkrijg satellietbeelden van deze exacte coördinaten voor verschillende drempels.

Nu moeten we satellietbeelden downloaden voor verschillende NDVI-drempels.
Daarna kunnen we de NDVI-maskers vergelijken met de gelabelde maskers. We zullen de MSE gebruiken om de fout tussen twee afbeeldingen te berekenen.

In [ ]:
# Import necessary libraries
from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient,DataCollection, bbox_to_dimensions, SHConfig
# configuration sentinel hub 
config = SHConfig()
#je mag mijn client credentials gebruiken
config.sh_client_id = '0bd9981e-9f55-48f8-a564-f5db87422602' 
config.sh_client_secret = '2#?ILN5#vU&0y6+{vm*]:^+Fqae^ro]:!>I1ka8+'

if config.sh_client_id == '' or config.sh_client_secret == '':
    print("Warning! To use Sentinel Hub Process API, please provide the credentials (client ID and client secret).")

In [ ]:
slots = [('2017-01-01', '2018-01-01'),
('2018-01-01', '2019-01-01'),
('2019-01-01', '2020-01-01'),
('2020-01-01', '2020-12-31')]

In [ ]:
evalscript_ndvi_values = """

//VERSION=3
    function setup() {
      return{
        input: [{
          bands: ["B04", "B08"]
        }],
        output: {
          id: "default",
          bands: 1,
          sampleType: SampleType.FLOAT32
        }
      }
    }

    

    function evaluatePixel(sample) {
      let ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04)
      return [ ndvi ]
    }
"""

In [ ]:
no_clouds = { 
    "dataFilter": { 
        "maxCloudCoverage": 0
    } 
}

In [ ]:
def plot_image(image, factor=1/255, clip_range=(0, 1), **kwargs):
    """
    Utility function for plotting RGB images.
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])
    
def send_request(evalscript, input_data, coords, config, save_data, data_folder='../data/images'):
    bbox = BBox(bbox=coords, crs=CRS.WGS84)
    request = SentinelHubRequest(data_folder=data_folder, evalscript=evalscript, input_data=input_data, bbox=bbox, config=config, size=[512, 512], responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)])
    response = request.get_data(save_data=save_data)[0]
    return response

# def send_request_geojson(evalscript, input_data, coords, config, save_data, data_folder='../data/images'):
#     bbox = BBox(bbox=coords, crs=CRS.WGS84)
#     request = SentinelHubRequest(data_folder=data_folder, evalscript=evalscript, input_data=input_data, bbox=bbox, config=config, size=[512,512], responses=[SentinelHubRequest.output_response('default', MimeType.JSON)])
#     response = request.get_data(save_data=save_data)[0]
#     return response

simple_request = lambda eval_script, coords, config, save_data, **kwargs: send_request(
    eval_script, 
    [SentinelHubRequest.input_data(data_collection=DataCollection.SENTINEL2_L2A, time_interval=('2018-01-01', '2019-01-01'), **kwargs)], 
    coords, config, save_data
)

# Function that sends a request for an satellite image of a scene and timestamp 
sentinel_request = lambda eval_script, coords, time_interval, config, save_data, data_folder, **kwargs: send_request(
    eval_script, 
    [SentinelHubRequest.input_data(data_collection=DataCollection.SENTINEL2_L2A, time_interval=time_interval, **kwargs)],
    coords, config, save_data, data_folder)

In [ ]:
# zet dit in python script - reduce download mask function 
def generate_mask(coords, slot, threshold):
    img_ndvi = sentinel_request(evalscript_ndvi_values, coords,slot, config, False,'/content/drive/My Drive/BB8/West_Bengal_15_geolocations_v2/junk_images', other_args=no_clouds)
    min_ndvi = threshold
    ndvi_copy = img_ndvi.copy()
    img_arr = np.where(ndvi_copy > min_ndvi, 255, 0)
    img_arr = img_arr.astype(np.uint8)
    # img.show()
    return img_arr

In [ ]:
#coordinates of the selected labeled images.
coords_ = [[87.000687,21.876816, 87.04768700000001, 21.923816], [87.00763543888888,21.94499273055556,87.0546354388889,21.991992730555555], 
           [87.007093,21.858627000000002,87.0540930000000,21.905627], [87.01661899999999,21.898952,87.063619,21.945952]]

voor elk  bounding box in coords_:
* true color afbeeldingen weergeven
* gelabeld masker weergeven
* NDVI-afgeleide maskers weergeven voor verschillende drempels variërend van 0,5 tot 1,0 met een stap van 0,05

In [ ]:
def mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err

  # bron (moet in OR komen te staan vgm- https://www.pyimagesearch.com/2014/09/15/python-compare-two-images/ )

In [ ]:

def plot_masks(bbox, labeled_mask, slot):
    print(f" These masks are generated for bbox: {bbox}")
    
    fig = plt.figure(figsize=(50,50))
    rows, columns= 5, 5
    #Adds a subplot at the 1st position
    fig.add_subplot(rows, columns, 1)

  #   showing image
    labeled_mask_img =  Image.fromarray(labeled_mask)
    plt.imshow(labeled_mask_img)
    plt.axis('off')
    plt.title("Labeleld images")

  # Iterate  throught the diferent thresholds
    count= 2
    for threshold in np.arange(0.1, 0.6,0.05):
      fig.add_subplot(rows, columns, count)
      count +=1
      # get image array of masks from Sentinel hub with given threshold
      img_arr = generate_mask(bbox, slot, threshold)
      plt.imshow(Image.fromarray(img_arr))
      plt.title(f'NDVI threshold:{threshold}' + '\n'+f'MSE: {mse(labeled_mask, img_arr)}', fontsize= 20)
      

In [ ]:
plot_masks(coords_[0], labeled_masks[0], slots[2])

In [ ]:
plot_masks(coords_[1],labeled_masks[1], slots[2] )

In [ ]:
plot_masks (coords_[2], labeled_masks[2], slots[3])

In [ ]:
plot_masks(coords_[3], labeled_masks[3], slots[3])

## Check voor welke drempel de minste fouten van het masker 2020 geven in vergelijking met de andere maskers.

In [ ]:
# get first cooridnate from cooridinate

locations = df = pd.read_csv('/content/drive/My Drive/BB8/Converted.csv')
locations.iloc[:3]

In [ ]:
from sentinelhub.geo_utils import to_wgs84

def bbox_converter(x):
    """
    Convert coordinates (longitude, latitude) in to a bbox in WGS84 format.
    
    return [long, lat, long, lat]
    """
    lng, lat = to_wgs84(x['Longitude'], x['Latitude'], CRS.WGS84)
    coords = [lng -  0.025, lat - 0.025, lng +  0.022, lat + 0.022]
    return coords

In [ ]:
# I wanted to see the NDVI derived masks with an threshold of 0.45 for the year 2017 and 2018.
mask_ndvi_1= generate_mask(bbox_converter(locations.iloc[0]), slots[0], 0.45)
mask_ndvi_2= generate_mask(bbox_converter(locations.iloc[0]), slots[1], 0.45)
mask_ndvi_3= generate_mask(bbox_converter(locations.iloc[0]), slots[2], 0.45)
mask_ndvi_4= generate_mask(bbox_converter(locations.iloc[0]), slots[3], 0.4)
masks = [mask_ndvi_1, mask_ndvi_2, mask_ndvi_3]
display(Image.fromarray(mask_ndvi_4))
[display(mse(mask , mask_ndvi_4)) for mask in masks]

In [ ]:
fig = plt.figure(figsize=(50,50))
rows, columns= 5, 5
#Adds a subplot at the 1st position
for i in range(len(masks)+1):
  if i != len(masks):
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(masks[i])
    Image.fromarray(masks[i]).save(f'/content/drive/My Drive/BB8/West_Bengal_15_geolocations_v2/junk_images/img{i}.png')
    plt.title(f"{slots[i]}")
    plt.title(f"{slots[i]} and mse with the last one {mse(masks[i] , mask_ndvi_4)}")

  else:
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(mask_ndvi_4)
    Image.fromarray(mask_ndvi_4).save(f'/content/drive/My Drive/BB8/West_Bengal_15_geolocations_v2/junk_images/img{i}.png')
    plt.title(f"{slots[i]} ")


In [ ]:
# I wanted to see the NDVI derived masks with an threshold of 0.45 for the year 2017 and 2018.
random_int = np.random.randint(15)
print(random_int)
mask_ndvi_1= generate_mask(bbox_converter(locations.iloc[random_int]), slots[0], 0.45)
mask_ndvi_2= generate_mask(bbox_converter(locations.iloc[random_int]), slots[1], 0.45)
mask_ndvi_3= generate_mask(bbox_converter(locations.iloc[random_int]), slots[2], 0.45)
mask_ndvi_4= generate_mask(bbox_converter(locations.iloc[random_int]), slots[3], 0.45)
masks = [mask_ndvi_1, mask_ndvi_2, mask_ndvi_3]
display(Image.fromarray(mask_ndvi_4))
[display(mse(mask , mask_ndvi_4)) for mask in masks]

fig = plt.figure(figsize=(50,50))
rows, columns= 5, 5
#Adds a subplot at the 1st position
for i in range(len(masks)+1):
  if i != len(masks):
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(masks[i])
    Image.fromarray(masks[i]).save(f'/content/drive/My Drive/BB8/West_Bengal_15_geolocations_v2/junk_images/img2{i}.png')
    plt.title(f"{slots[i]}")
    plt.title(f"{slots[i]} and mse with the last one {mse(masks[i] , mask_ndvi_4)}")

  else:
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(mask_ndvi_4)
    Image.fromarray(mask_ndvi_4).save(f'/content/drive/My Drive/BB8/West_Bengal_15_geolocations_v2/junk_images/img2{i}.png')
    plt.title(f"{slots[i]} ")


# Conclusie


With the current results masks and MSE's for the different NDVI thresholds, we can come to the followin conclusion: <br><br>
IMG 1 and 2 for given time window (01-01-20219, 01-01-2020) <br><br>
Image 1 <br>

| NDVI threshold | Error |
|-----|-----|
|  0.4  | 11520  |
|  0.45 |  10103  |
|  0.5  |  11596  |
|  0.55 |  14606 |

<br><br>
Image 2<br>

| NDVI threshold | Error |
|-----|-----|
|  0.4  | 15139 |
|  0.45 | 10765 |
|  0.5  | 10181 |
|  0.55 | 12039 |

<br><br>
IMG3 and 4 for given time window (01-01-2020, 31-12-2020) <br><br>
Image 3 <br>

| NDVI threshold | Error |
|-----|-----|
|  0.4  | 9828  |
|  0.45 |  11227  |
|  0.5  |  14955  |
|  0.55 |  19363 |

<br><br>
Image 4<br>

| NDVI threshold | Error |
|-----|-----|
|  0.4  | 11303  |
|  0.45 |  9421  |
|  0.5  |  9184  |
|  0.55 |  10458 |

Aan de hand van deze resultaten is te zien dat de optimale globale threshold is 0.45. Bovendien is  er te zien dat er veel errors zitten tussen de NDVI masks en de gelabelde masks zijn. Er is ter zien dat het lastig isom bebossing te onderscheiden in satellietbeelden aan de hand van puur NDVI waardes.

Door tijdsgebrek gaan we dat een globale threshold van 0.55 gebruiken voor onze masks. Bovendien gaan we 2020 weghalen want die is heel inconsistent met de anderen jaren. Een reden voor deze inconsistente masks zijn namelijk de *fade*
in de satellietbeelden.